# Preprocess ICRP-107 to create dataset for radioactivedecay
### Introduction
This notebook creates a decay dataset for `radioactivedecay` from the data in <a href="http://www.icrp.org/publication.asp?id=ICRP%20Publication%20107">ICRP 107: Nuclear Decay Data for Dosimetric Calculations</a>.

The data is saved into the following files:
- `decay_data.npz`: NumPy arrays containing the strings of the radionuclides in the decay dataset, the half-lives and their time units, the atomic masses, the progeny, branching fractions and decay modes of each radionuclide, and the days to year conversion number (NPZ NumPy compressed array format). Atomic data of the stable nuclides is included from the <a href="https://iopscience.iop.org/article/10.1088/1674-1137/abddb0/meta">AME2020</a> database, with supplemental metastable isomer excitation energies from <a href="https://iopscience.iop.org/article/10.1088/1674-1137/abddae">NuBase</a>;
- `c_scipy.npz`: pre-calculated SciPy CSR sparse matrix *C* (Amaku et al. (2010)) (NPZ NumPy compressed array format);
- `c_inv_scipy.npz`: pre-calculated SciPy CSR sparse matrix *C<sup>-1</sup>* (inverse of *C*) (NPZ NumPy compressed array format);
- `c_sympy.pickle`: pre-calculated SymPy sparse matrix *C* for arbitrary-precision calculations (Python pickle format);
- `c_inv_sympy.pickle`: pre-calculated SymPy sparse matrix *C<sup>-1</sup>* for arbitrary-precision calculations (inverse of *C*) (Python pickle format);
- `atomic_masses_sympy.pickle`: SymPy matrix of atomic masses(g/mol)for arbitrary-precision calculations (Python pickle format);
- `decay_consts_sympy.pickle`: SymPy matrix of radionuclide decay constants (s<sup>-1</sup>) for arbitrary-precision calculations (Python pickle format);
- `year_conversion_sympy.pickle`: SymPy representation of the days to year conversion number (Python pickle format).

### Initial set up and read in ICRP-107 data into a DataFrame 
First load the necessary Python modules.

In [34]:
import itertools, pickle, re
import fortranformat as ff
import numpy as np
import pandas as pd
from scipy import sparse
from sympy import Integer, log, Matrix, nsimplify, S
from sympy.matrices import SparseMatrix
import json as json

### Define helper functions

In [35]:
year = 365.2422

def to_rational(number):
    """
    Converts string representation of a number to a SymPy object.
    """

    if 'e' in number or 'E' in number:
        if 'e' in number:
            end = number.split('e')[1]
            number = number.split('e')[0]
        else:
            end = number.split('E')[1]
            number = number.split('E')[0]
        parts = number.split('.')
        if len(parts) == 1: parts.append('')
        if end[0] == '+' or end[0].isdigit():
            multiply = 1
            factor = S(10**int(end.lstrip('+')))
        else:
            multiply = 0
            factor = S(10**int(end.lstrip('-')))
        denom = S(10**len(parts[1]))
        parts[0] = parts[0].lstrip('0')
        if len(parts[0]) == 0: parts[1] = parts[1].lstrip('0')
        if multiply == 1:
            return S(parts[0]+parts[1])*factor/denom
        else: return S(parts[0]+parts[1])/(denom*factor)
    parts = number.split('.')
    if len(parts) == 1: parts.append('')
    denom = S(10**len(parts[1]))
    parts[0] = parts[0].lstrip('0')
    if len(parts[0]) == 0: parts[1] = parts[1].lstrip('0')
    return S(parts[0]+parts[1])/denom

### Read the nuclide data

In [36]:
with open('nuclides.json') as nuclides_json:
    nuclides = json.load(nuclides_json)

nuclide_list = list(nuclides.keys())

### Make the *&Lambda;* matrix
Now we make the sparse lower triangular matrix *&Lambda;*, which captures the decay pathways and branching relations between the radionuclides. _&Lambda;_ is set up based on Eq. (6) in Amaku et al. (2010). The diagonal elements are all *-&lambda;<sub>jj</sub>*, i.e. negative decay constant for each radionuclide. The off-diagonal elements are all of the form *BF<sub>ij</sub>&times;&lambda;<sub>jj</sub>* for *i* > *j*, where *BF<sub>ij</sub>* is the branching fraction from parent *j* to progeny *i*. The non-zero elements beneath the *jj*<sup>th</sup> element in each column are first progeny of radionuclide *j*.

In [37]:
rows = np.array([], dtype=np.int64)
cols = np.array([], dtype=np.int64)
data = np.array([], dtype=np.float64)
ln2 = np.log(2)

for parent in nuclide_list:
    j = nuclide_list.index(parent)
    rows = np.append(rows, [j])
    cols = np.append(cols, [j])
    half_life = nuclides[parent]['half_life']
    if half_life == None:
        lambd = 0.0
    else:
        lambd = ln2/half_life
    data = np.append(data, -lambd)
    for progeny in nuclides[parent]['daughters']:
        if (nuclides[progeny]['stable']): continue
        if (progeny not in nuclide_list): continue
        i = nuclide_list.index(progeny)
        rows = np.append(rows, [i])
        cols = np.append(cols, [j])
        data = np.append(data, [lambd*float(nuclides[parent]['daughters'][progeny])])

lambda_mat = sparse.csc_matrix((data, (rows, cols)))

### Calculate the matrices *C* and _C<sup>-1</sup>_
We now need to make the sparse matrices *C* and *C<sup>-1</sup>*, which are given by Eqs. (10) and (13) in Amaku et al. (2010), respectively. The diagonal elements of both matrices are 1. *C* and *C<sup>-1</sup>* differ from *&Lambda;* in that there are non-zero elements beneath the *jj*<sup>th</sup> element in each column for all progeny of *j*, i.e. everything in its full decay chain, not just the immediate daughters.

Therefore we have to find all the progeny in the decay chain of each radionuclide. We do this by looping backwards over each column in *&Lambda;* to build up lists of the radionuclides in the decay chain of each parent. We then set up the basic structure (i.e. define the non-zero elements) of sparse matrices *C* and *C<sup>-1</sup>*.

In [38]:
num_nuclides = len(nuclide_list)
# dictionary of decay chain members by index
rows_dict = {}
for i in range(num_nuclides-1, -1, -1):
    a,_ = lambda_mat[:,i].nonzero()
    b = a
    for j in a:
        if j > i:
            b = np.unique(np.concatenate((b,rows_dict[j])))
    rows_dict[i] = b

rows_c = np.array([], dtype=np.int32)
cols_c = np.array([], dtype=np.int32)
for i in range(0, num_nuclides):
    # row of C initialized as array of decay chain members
    rows_c = np.concatenate((rows_c,rows_dict[i]))
    # column of C initialized as array of index with length of chain size
    cols_c = np.concatenate((cols_c,np.array([i]*len(rows_dict[i]), dtype=np.int32)))

# initialize c and c^(-1) as sparse csc matrices with ones on diagonal
c = sparse.eye(num_nuclides, num_nuclides, dtype=np.float64, format="csc")
c_inv = sparse.eye(num_nuclides, num_nuclides, dtype=np.float64, format="csc")

Now calculate *C* and *C<sup>-1<sup>*. Note that only the non-zero elements of *C<sub>kj</sub>* and *C<sup>-1</sup><sub>kj</sub>*  need to be considered for the sums in Eqs. (10) and (13) of Amaku et al. (2010).
    
Calculate *C*, and highlight any cases where the relative difference of the decay constants of two radionuclides in the same decay chain is less than 0.001 (as this could lead to numerical precision issues):

In [39]:
for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: continue
    sigma = 0.0
    for k in rows_dict[j]:
        if k == i: break
        sigma += lambda_mat[i,k]*c[k,j]
    c[i,j] = sigma/(lambda_mat[j,j]-lambda_mat[i,i])

    if abs((lambda_mat[j,j]-lambda_mat[i,i])/lambda_mat[j,j]) < 1E-3: print(nuclide_list[i], nuclide_list[j])

C:\Users\dmnev\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\sparse\_index.py:100: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


There are no cases where radionuclides in the same decay chain have decay constants that are too similar (for the radionuclides in ICRP-107). Now proceed to calculate *C<sup>-1</sup>*:

In [40]:
for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: continue
    sigma = 0.0
    for k in rows_dict[j]:
        if k == i: break
        sigma -= c[i,k]*c_inv[k,j]
    c_inv[i,j] = sigma

### Calculate SymPy versions of the matrices for arbitrary-precision calculations
We now calculate SymPy versions of *C* and *C<sup>-1<sup>* for arbitrary-precision calculations. First define some functions for processing the data into SymPy objects:

In [41]:
year_sympy = S(3652422)/10000

def convert_half_life_sympy(halflife, unit):
    """
    Conversion of SymPy half-life into seconds.
    """

    units = {'μs':S(1)/1000000,
             'ms':S(1)/1000,
             's':S(1),
             'm':S(60),
             'h':S(3600),
             'd':S(86400),
             'y':S(86400)*year_sympy
            }
    return halflife*units[unit]

Now make a SymPy version of the *&Lambda;* matrix:

In [42]:
lambda_mat_sympy = SparseMatrix.zeros(num_nuclides, num_nuclides)
lambdas_sympy = Matrix.zeros(num_nuclides, 1)
masses_sympy = Matrix.zeros(num_nuclides, 1)

for parent in nuclide_list:
    j = nuclide_list.index(parent)
    if nuclides[parent]['half_life'] == None:
        lambd = Integer(0)
        lambda_mat_sympy[j, j] = Integer(0)
        lambdas_sympy[j] = Integer(0)
    else:
        hl_sympy = nuclides[parent]['half_life']
        lambd = log(2)/hl_sympy
        lambda_mat_sympy[j, j] = -lambd
        lambdas_sympy[j] = lambd
    for progeny in nuclides[parent]['daughters']:
        if (nuclides[progeny]['stable']): continue
        if (progeny not in nuclide_list): continue
        i = nuclide_list.index(progeny)
        lambda_mat_sympy[i, j] = lambd*nuclides[parent]['daughters'][progeny]
    masses_sympy[j] = 0

Now make a SymPy version of the *C* and *C<sup>-1</sup>* matrix:

In [43]:
c_sympy = SparseMatrix.eye(num_nuclides)
c_inv_sympy = SparseMatrix.eye(num_nuclides)

for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: continue
    sigma = Integer(0)
    for k in rows_dict[j]:
        if k == i: break
        sigma += lambda_mat_sympy[i,k]*c_sympy[k,j]
    c_sympy[i,j] = sigma/(lambda_mat_sympy[j,j]-lambda_mat_sympy[i,i])

for index in range(0, rows_c.size):
    i = rows_c[index]
    j = cols_c[index]
    if i == j: continue
    sigma = Integer(0)
    for k in rows_dict[j]:
        if k == i: break
        sigma -= c_sympy[i,k]*c_inv_sympy[k,j]
    c_inv_sympy[i,j] = sigma 

### Save the outputs
Write out file containing NumPy arrays with the radionuclide names, the half-lives, their time units, human-readable half-life strings, the progeny, branching fractions and decay modes of each radionuclide, the atomic masses of both unstable and stable nuclides, and the days to year conversion number. Write output files containing *C* and *C<sup>-1</sup>* in the SciPy and SymPy sparse formats. Write out the SymPy versions of the decay constants.

In [44]:
half_lives = []
half_life_units = []
half_life_readable = []
for nuclide in nuclide_list:
    half_lives.append(nuclides[nuclide]['half_life'])
    half_life_units.append('s')
    half_life_readable.append(True)
hldata = np.array([(np.float64(a),b,c) for a, b, c in zip(half_lives, half_life_units, half_life_readable)], dtype=object)

progeny = [[] for _ in range(0, len(nuclide_list))]
bfs = [[] for _ in range(0, len(nuclide_list))]
modes = [[] for _ in range(0, len(nuclide_list))]
for i in range(0, len(nuclide_list)):
    prog, bf, mode = [], [], []
    for d in nuclides[nuclide_list[i]]["daughters"]:
        prog.append(d)
        bf.append(float(nuclides[nuclide_list[i]]["daughters"][d]))
        mode.append('beta')
    if len(mode) > 0:
        # sort by decreasing branching fraction
        triple = list(zip(bf, prog, mode))
        triple.sort(key=lambda t: (-t[0], t[1], t[2]))
        bfs[i], progeny[i], modes[i] = map(list, zip(*triple))

np.savez_compressed("./decay_data.npz", nuclides=np.asarray(nuclide_list),
                    masses=masses_sympy,
                    hldata=hldata, progeny=np.asarray(progeny, dtype=object),
                    bfs=np.asarray(bfs, dtype=object), modes=np.asarray(modes, dtype=object),
                    year_conv=np.float64(year))

# Write out SciPy sparse matrices (convert to CSR format)
sparse.save_npz("./c_scipy.npz", c.tocsr())
sparse.save_npz("./c_inv_scipy.npz", c_inv.tocsr())

import pkg_resources, sympy
if pkg_resources.parse_version(sympy.__version__) >= pkg_resources.parse_version('1.9'):
    pickle_type = '1.9'
else:
    pickle_type = '1.8'

# Write out SymPy objects to pickle files
with open(f"c_sympy_{pickle_type}.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(c_sympy))
with open(f"c_inv_sympy_{pickle_type}.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(c_inv_sympy))
with open(f"atomic_masses_sympy_{pickle_type}.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(masses_sympy))
with open(f"decay_consts_sympy_{pickle_type}.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(lambdas_sympy))
with open(f"year_conversion_sympy_{pickle_type}.pickle", "wb") as outfile:
    outfile.write(pickle.dumps(year_sympy))